In [ ]:

!git clone https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
!git reset --hard
!git pull 

In [ ]:
!pip3 uninstall torch torchvision torchaudio
!pip uninstall transformers
!pip uninstall accelerate -U


In [ ]:
import os
os.chdir('LMKBC-2023/')

In [ ]:
!pip install transformers
!pip install accelerate -U


In [ ]:
!python src/run.py --mode "train_fm"



In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!git add . 
!git commit -m 'from server' 
!git push


In [ ]:
#!model_load_dir = '/workspace/persistent/LMKBC-2023/bin/pretrain_fill_mask/bert-large-cased/best_ckpt'
!model_load_dir ='/workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased/best_ckpt'

!python  /workspace/persistent/LMKBC-2023/src/fm_model.py  --test_fn /workspace/persistent/LMKBC-2023/data/val.jsonl --template_fn res/prompts0.csv   --output_fn /workspace/persistent/LMKBC-2023/output/filled-mask-valid.jsonl --train_fn /workspace/persistent/LMKBC-2023/data/train.jsonl          --train_batch_size 256 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  /workspace/persistent/LMKBC-2023/data/train_tiny.jsonl         --mode "tr ain test"  --train_epoch 50 --learning_rate 5e-5         --model_load_dir   $model_load_dir         --model_save_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased        --model_best_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased/best_ckpt 

In [ ]:
   python {config.SRC_DIR}/fm_model.py  --test_fn {test_fn} --template_fn res/prompts0.csv  --output_fn {OUTPUT_FILE} --train_fn {config.DATA_DIR}/train.jsonl --train_batch_size 64 --gpu {args.gpu}  --top_k 30 --threshold 0.1  --dev_fn  {config.DATA_DIR}/train_tiny.jsonl --mode "train test" --train_epoch 50 --learning_rate 5e-5 --model_load_dir {last_best_dir} --model_save_dir {fm_model_save_dir} --model_best_dir  {fm_model_best_dir}

In [ ]:
!nvidia-smi

In [ ]:
!cat nohup.out 

In [ ]:
  !python /workspace/persistent/LMKBC-2023/src/fm_model.py  --test_fn /workspace/persistent/LMKBC-2023/data/val.jsonl --template_fn res/prompts0.csv  --output_fn /workspace/persistent/LMKBC-2023/output/filled-mask-valid.jsonl --train_fn /workspace/persistent/LMKBC-2023/data/train.jsonl --train_batch_size 64 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  /workspace/persistent/LMKBC-2023/data/train_tiny.jsonl --mode "train test" --train_epoch 50 --learning_rate 5e-5 --model_load_dir /workspace/persistent/LMKBC-2023/bin/default/pretrain_fill_mask/best_ckpt --model_save_dir /workspace/persistent/LMKBC-2023/bin/default/fill_mask --model_best_dir  /workspace/persistent/LMKBC-2023/bin/default/fill_mask/best_ckpt

In [ ]:
!tail -n 1000  nohup.out| grep loss

In [9]:
import os
import json 
import sys
print('cwd', os.getcwd())
import pandas as pd
sys.path.append('src')
import src.util


result_fp = 'res/results.jsonl'
sentence_list =  open(result_fp).readlines()
json_list = [json.loads(s) for s in sentence_list]
result_list=[]
for json_dict in json_list:
    model_load_dir = json_dict['args']['model_load_dir']
    if len(model_load_dir) < 20:
        label = model_load_dir
    else:
        label = model_load_dir.split('/')[1]
    epoch = json_dict['args']['train_epoch']
    if epoch <30:
        continue
    #print(label, epoch )
    need_keys = ['CountryBordersCountry','CountryHasOfficialLanguage',
                 'CountryHasStates',"StateBordersState",'Average']
    metric = json_dict['metric']
    result = dict()
    result['label'] = label
    #print(result.keys())
    for k in need_keys:
        result[k] = metric[k]['best_f1']
    result_list.append(result)
    
scores_per_relation_pd = pd.DataFrame(result_list)
print(scores_per_relation_pd.round(3).to_string(max_cols=12))
#print(scores_per_relation_pd.transpose().round(2).to_string(max_cols=12))


cwd /workspace/project/LMKBC-2023
               label  CountryBordersCountry  CountryHasOfficialLanguage  CountryHasStates  StateBordersState  Average
0          workspace                  0.000                       0.000             0.000              0.000    0.000
1          workspace                  0.357                       0.568             0.028              0.013    0.246
2   bert-large-cased                  0.358                       0.585             0.022              0.012    0.248
3          workspace                  0.380                       0.506             0.097              0.013    0.252
4               home                  0.370                       0.587             0.007              0.017    0.247
5    bert-base-cased                  0.365                       0.581             0.019              0.014    0.247
6    bert-base-cased                  0.000                       0.000             0.000              0.000    0.000
7    bert-base-cased  